In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

import pickle

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [3]:
import mlflow

# setting tracking uri as sqlite backend and also experiment name 

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_experiment")

<Experiment: artifact_location=('/mnt/e/Machine_Learning_Projects/mlops_zoomcamp/02. Experiment '
 'Tracking/mlruns/1'), creation_time=1748346800773, experiment_id='1', last_update_time=1748346800773, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

#### running the same trip duration experiment but including mlops in it 

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

In [6]:
df_train.shape, df_val.shape

((73908, 21), (61921, 21))

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715209663881

In [13]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [9]:
# now we will log it in mlflow for Lasso Regression model 

with mlflow.start_run():

    mlflow.set_tag("developer", "Arun")

    mlflow.log_param("train_data_path", "green_tripdata_2021-01.parquet")
    mlflow.log_param("val_data_path", "green_tripdata_2021-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)

    lr = Lasso()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # this will be just recorded as artifact like a files in mlflow 
    #mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")
    mlflow.sklearn.log_model(lr, artifact_path="ml_models")
    

2025/05/28 19:47:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
# using model logged from mlflow run to predict 

import mlflow
logged_model = 'runs:/33857249a18c41bab5f71e6edf1b4826/ml_models'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

values = loaded_model.predict(X_train)
values

array([16.84934773, 16.84946739, 16.84935639, ..., 16.84946503,
       16.85040816, 16.84941071], shape=(73908,))

#### using hyper paramter tuning on xgboost model and mlflow logging together 

In [21]:
import xgboost as xgb 

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [22]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
# objective function to get the best params based on defined metrics (optimization)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=50,
            evals=[(valid, 'validation')],
            early_stopping_rounds=30
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [24]:
# search space to search the hyper parameters

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'seed': 42
}

# now passing the objective and search space to run the algorithm 
best_params = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=15,
    trials=Trials()
)

print(best_params)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

[0]	validation-rmse:8.37069                           
[1]	validation-rmse:7.19566                           
[2]	validation-rmse:6.85475                           
[3]	validation-rmse:6.73971                           
[4]	validation-rmse:6.70209                           
[5]	validation-rmse:6.68342                           
[6]	validation-rmse:6.67210                           
[7]	validation-rmse:6.66530                           
[8]	validation-rmse:6.66148                           
[9]	validation-rmse:6.65665                           
[10]	validation-rmse:6.65196                          
[11]	validation-rmse:6.64989                          
[12]	validation-rmse:6.64779                          
[13]	validation-rmse:6.64397                          
[14]	validation-rmse:6.64142                          
[15]	validation-rmse:6.63750                          
[16]	validation-rmse:6.63625                          
[17]	validation-rmse:6.63439                          
[18]	valid